# Pumped-Storage Optimisation with Neural Network trained by Genetic Algorithm

In [1]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_ANN, evaluate_fitness

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

# Import Pytorch stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

# Import genetic algorithm library
from deap import base, creator, tools, algorithms

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## The Power Plant

In [2]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]
plant_params["LOOKAHEAD"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * 3

## Price Data

In [3]:
df = pd.read_csv("../01 - Data/spot_prices_utc.csv").assign(utc_time = lambda x: pd.to_datetime(x.utc_time))
df.head(2)

,spot,utc_time
0,36.99,2017-12-31 23:00:00+00:00
1,31.08,2018-01-01 00:00:00+00:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48166 entries, 0 to 48165
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   spot      48166 non-null  float64            
 1   utc_time  48166 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 752.7 KB


In [5]:
df.utc_time.min()

Timestamp('2017-12-31 23:00:00+0000', tz='UTC')

In [6]:
df.utc_time.max()

Timestamp('2023-06-30 21:00:00+0000', tz='UTC')

Keep the part that was before the madness:

In [7]:
df = df.query('utc_time < @pd.Timestamp("2020-01-01", tz="UTC")')

Then set two weeks aside, one for validation and one for testing.

In [8]:
df_train = df.query('utc_time < @pd.Timestamp("2019-12-18", tz="UTC")').reset_index(
    drop=True
)
df_val = (
    df.query('utc_time >= @pd.Timestamp("2019-12-18", tz="UTC")')
    .query('utc_time < @pd.Timestamp("2019-12-25", tz="UTC")')
    .reset_index(drop=True)
    .reset_index(drop=True)
)
df_test = df.query('utc_time >= @pd.Timestamp("2019-12-25", tz="UTC")').reset_index(
    drop=True
)

In [9]:
print(df_train.shape, df_val.shape, df_test.shape)

(17184, 2) (168, 2) (168, 2)


### Preparing the dataset for the neural network

In [10]:
df_train

,spot,utc_time
0,36.99,2017-12-31 23:00:00+00:00
1,31.08,2018-01-01 00:00:00+00:00
2,29.17,2018-01-01 01:00:00+00:00
3,21.96,2018-01-01 02:00:00+00:00
4,14.96,2018-01-01 03:00:00+00:00
...,...,...
17179,47.55,2019-12-17 19:00:00+00:00
17180,41.18,2019-12-17 20:00:00+00:00
17181,41.53,2019-12-17 21:00:00+00:00
17182,39.84,2019-12-17 22:00:00+00:00


Create maximum number of possible 168 hour sequences and put them into table format:

In [11]:
def create_sequences(in_array, sequence_length):
    out_array = np.zeros((len(in_array)-sequence_length+1, sequence_length), dtype=np.float32)

    for i in tqdm(range(len(in_array)-sequence_length+1)):
        out_array[i] = np.array(in_array[i:i+sequence_length])

    return out_array

In [12]:
len(df_train.spot)

17184

In [13]:
X_train = create_sequences(df_train.spot, 168)

  0%|          | 0/17017 [00:00<?, ?it/s]

In [14]:
X_train

array([[36.99, 31.08, 29.17, ..., 42.58, 45.05, 33.3 ],
       [31.08, 29.17, 21.96, ..., 45.05, 33.3 , 21.95],
       [29.17, 21.96, 14.96, ..., 33.3 , 21.95, 19.67],
       ...,
       [42.43, 37.36, 34.94, ..., 47.55, 41.18, 41.53],
       [37.36, 34.94, 33.57, ..., 41.18, 41.53, 39.84],
       [34.94, 33.57, 32.8 , ..., 41.53, 39.84, 38.81]], dtype=float32)

The different columns represent the hour 1 to hour 168, but indexed at zero, hence only until 167. Each row will be fed into the neural network.

In [15]:
pd.DataFrame(X_train).head(3)

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
0,36.990002,31.080000,29.170000,21.959999,14.96,-5.92,-6.10,-10.02,-5.71,-1.88,...,20.660000,20.299999,24.150000,32.270000,46.389999,46.919998,45.450001,42.580002,45.049999,33.299999
1,31.080000,29.170000,21.959999,14.960000,-5.92,-6.10,-10.02,-5.71,-1.88,-10.48,...,20.299999,24.150000,32.270000,46.389999,46.919998,45.450001,42.580002,45.049999,33.299999,21.950001
2,29.170000,21.959999,14.960000,-5.920000,-6.10,-10.02,-5.71,-1.88,-10.48,-6.79,...,24.150000,32.270000,46.389999,46.919998,45.450001,42.580002,45.049999,33.299999,21.950001,19.670000


## MILP

Find the solutions for the validation and testing periods.

In [16]:
milp_solver = MILP(plant_params=plant_params, spot=df_val["spot"], utc_time=df_val["utc_time"])
milp_model, milp_status, milp_profile = milp_solver.solve()
print(milp_status)
optimal_val = milp_model.objective.value()
print(optimal_val)
milp_profile.head(2)

Optimal
46943.0


,water_level,action,colour_id,utc_time,spot
0,375.0,-1,pump,2019-12-18 00:00:00+00:00,35.45
1,750.0,-1,pump,2019-12-18 01:00:00+00:00,34.29


In [17]:
milp_solver = MILP(plant_params=plant_params, spot=df_test["spot"], utc_time=df["utc_time"])
milp_model, milp_status, milp_profile = milp_solver.solve()
print(milp_status)
optimal_test = milp_model.objective.value()
print(optimal_test)
milp_profile.head(2)

Optimal
43665.0


,water_level,action,colour_id,utc_time,spot
0,375.0,-1,pump,2017-12-31 23:00:00+00:00,7.50
1,750.0,-1,pump,2018-01-01 00:00:00+00:00,5.47


## Feed Forward Neural Network

Predict the full week (168 output neurons, sigmoid activation to output values between 0 and 1, then use thresholds for translations into actions).

Randomly initiate the lake level such that the network can learn different situations.

In [18]:
class ANN(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, hidden_size):
        super(ANN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layers = hidden_layers
        self.hidden_size = hidden_size
        
        # Define input layer
        self.input_layer = nn.Linear(input_size, hidden_size)
        
        # Define hidden layers
        self.hidden_layers = nn.ModuleList()
        for _ in range(hidden_layers):
            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
        
        # Define output layer
        self.output_layer = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

In [19]:
model = ANN(input_size=168, output_size=168, hidden_layers=1, hidden_size=16)
model

ANN(
  (input_layer): Linear(in_features=168, out_features=16, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=16, out_features=16, bias=True)
  )
  (output_layer): Linear(in_features=16, out_features=168, bias=True)
)

In [20]:
def encode_chromosome(model):
    """
    Encodes the model parameters into a single chromosome, which can be used for genetic algorithms.
    """
    chromosome = torch.tensor([])

    for param in model.parameters():
        chromosome = torch.cat((chromosome, param.view(-1)))

    return chromosome


def decode_chromosome(model, chromosome):
    """
    Decodes the chromosome into the model parameters and updates the model with the new parameters.
    """
    model_params = list(model.parameters())
    start = 0

    for param in model_params:
        end = start + torch.numel(param)
        param.data = chromosome[start:end].view(param.size())
        start = end

In [21]:
encode_chromosome(model)

tensor([ 0.0662,  0.0161,  0.0503,  ..., -0.1663, -0.0011,  0.1594],
       grad_fn=<CatBackward0>)

In [22]:
# decode_chromosome(model, torch.zeros(len(encode_chromosome(model))))
# list(model.parameters())

Experimentally setting the model parameters back to zero shows that the update of the neural net actually works.

In [23]:
next(model.parameters()).dtype

torch.float32

In [24]:
X_train[0].dtype

dtype('float32')

In [25]:
model(torch.tensor(X_train[0]))

tensor([1.0740e-01, 9.9997e-01, 7.3425e-05, 9.9863e-01, 1.9378e-05, 1.7169e-02,
        1.0000e+00, 2.1994e-01, 1.3855e-04, 3.3521e-02, 9.4192e-03, 4.0199e-03,
        9.8805e-01, 9.9182e-01, 9.1204e-02, 7.1242e-02, 8.3793e-02, 4.3274e-05,
        1.0000e+00, 5.4321e-02, 4.9813e-02, 6.5045e-01, 6.8753e-05, 1.2364e-02,
        1.6451e-03, 2.8006e-06, 1.5201e-05, 9.6326e-01, 3.6667e-03, 9.9821e-01,
        8.0142e-01, 9.9692e-01, 9.9999e-01, 1.5059e-01, 9.6297e-01, 1.1957e-01,
        9.9508e-04, 8.2724e-06, 4.8164e-04, 6.5671e-01, 9.9991e-01, 2.0477e-02,
        9.9996e-01, 1.0376e-02, 7.4357e-04, 4.8232e-03, 1.1287e-02, 4.8484e-01,
        3.3853e-01, 6.5278e-03, 3.4396e-03, 9.9982e-01, 9.8547e-01, 1.0000e+00,
        2.4054e-01, 9.9902e-01, 9.9999e-01, 1.7243e-01, 6.7401e-01, 6.1795e-04,
        6.2571e-01, 3.7973e-02, 9.9971e-01, 5.7277e-03, 1.4486e-01, 2.9378e-02,
        7.8482e-07, 4.7924e-03, 8.0543e-02, 7.4817e-01, 9.9338e-01, 2.2978e-01,
        9.9667e-01, 2.9673e-06, 2.9766e-

In [35]:
evaluate_fitness(
    population=[model(torch.tensor(X_train[0])).detach().numpy()],
    ps_params=plant_params,
    spot_prices=X_train[0],
)

[-2059640.0015473366]

### Generalising from one to another week


In [ ]:
model = ANN(input_size=169, output_size=168, hidden_layers=1, hidden_size=16)

In [20]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

population, fitnesses, best_ind, history = ga_solver.train(
    config={
        "MUTPB": 1.0,
        "POP_SIZE": 100,
        "CXPB": 0.3,
        "INITIAL_MUTATION_RATE": 0.05,
        "FINAL_MUTATION_RATE": 0.005,
        "TOURNAMENT_SIZE": 4,
    },
    total_generations=500,
    tune_mode=False,
)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
history

,generation,best,average,worst
0,0,-293032.0,-9980398.028,-10460514.0
1,1,-297433.0,-9738494.206,-10389603.0
2,2,-243306.0,-9532594.974,-10409811.0
3,3,-161953.0,-9400322.812,-10345659.0
4,4,-153819.0,-9079379.224,-10440012.0
...,...,...,...,...
295,295,298162.0,-442471.250,-9750296.0
296,296,298162.0,-142067.478,-9744398.0
297,297,298162.0,-202000.952,-9749372.0
298,298,298162.0,-82016.506,-9722400.0


In [21]:
(
    pn.ggplot(
        data=(
            history.drop("worst", axis=1)
            .melt(id_vars="generation")
        ),
        mapping=pn.aes(x="generation", y="value", colour="variable"),
    )
    + pn.geom_line()
    + pn.theme(figure_size=[6, 3])
    # + pn.scale_y_log10()
)

NameError: name 'history' is not defined

### Elite Selection

In [23]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

population, fitnesses, best_ind, history = ga_solver.train(
    config={
        "MUTPB": 1,
        "POP_SIZE": 1000,
        "INITIAL_MUTATION_RATE": 0.05,
        "FINAL_MUTATION_RATE": 0.005,
        "INITIAL_EXPLORATION":0.8,
        "ELITISM": 0.2,
    },
    total_generations=500,
    tune_mode=False,
)

  0%|          | 0/500 [00:00<?, ?it/s]

In [24]:
history

,generation,best,average,worst
0,0,-222606.0,-9929118.061,-10546732.0
1,1,-184636.0,-9709979.027,-10436068.0
2,2,-147540.0,-9426174.036,-10419410.0
3,3,-157399.0,-9060452.455,-10466355.0
4,4,-149594.0,-8915066.782,-10507443.0
...,...,...,...,...
495,495,332047.0,-3320078.164,-9764185.0
496,496,332047.0,-3029849.953,-9738900.0
497,497,332047.0,-3288952.718,-9746401.0
498,498,332047.0,-3730213.806,-9758546.0
